CBOW无法建模长距离依赖，且忽略了单词的顺序  
RNN
- $ h_t = tanh(h_{t-1}W_h+x_tW_x+b)$
- BTT: Backpropagation Through Time  必须保存每个时刻的中间数据，内存使用量大、梯度不稳定
- Truncated BPTT: 将时间轴方向上过长的网络在合适的位置进行截断，从而创建多个小型网络，然后对截出来的小型网络执行误差反向传播  
困惑度
- 语言模型预测性能指标
- 预测正确词出现的概率的倒数

In [ ]:
import numpy as np

class RNN:
    def __init__(self,Wx,Wh,b):
        self.params = [Wx,Wh,b]
        self.grads = [np.zeros_like(Wx),np.zeros_like(Wh),np.zeros_like(b)]
        self.cache = None
    def forward(self,x,h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x,Wx) + b
        h_next = np.tanh(t)
        self.cache = (x, h_prev, h_next)
        return h_next
    def backward(self,dh_next):
        Wx,Wh,b = self.params
        x,h_prev,h_next = self.cache
        dt = dh_next * (1-h_next**2)
        db = np.sum(dt,axis=0)
        dWh = np.dot(h_prev.T,dt)
        dh_prev=np.dot(dt,Wh.T)
        dWx = np.dot(x.T,dt)
        dx = np.dot(dt,Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev
        
        
        

In [ ]:
class TimeRNN:
    def __init__(self,Wx,Wh,b,stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx),np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        self.h, self.dh = None, None
        self.stateful = stateful
    def set_state(self,h):
        self.h = h
    def reset_state(self):
        self.h = None
    
    def forward(self,xs):
        Wx, Wh, b = self.params
        N,T,D = xs.shape
        D,H = Wx.shape
        self.layers = []
        hs = np.empty((N,T,H),dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N,H),dtype='f')
        
        for t in range(T):
            layer = RNN (*self.params)
            self.h = self.layers.forward(xs[:,t,:],self.h)
            hs[:,t,:] = self.h
            self.layers.append(layer)
        return hs
    def backward(self,dhs):
        Wx, Wh, b = self.params
        N,T,H = dhs.shape
        D,H=Wx.shape
        dxs=np.empty((N,T,D),dtype='f')
        dh=0
        grads=[0,0,0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx,dh=layer.bacward(dhs[:,t,:]+dh)
            dxs[:,t,:]=dx
            for i ,grad in enumerate(layer.grads):
                grads[i] += grad
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        return dxs
    
    

In [ ]:
import sys
sys.path.append("..")
import numpy as np
from common.time_layers import *


class SimpleRNNlm:
    def __init__(self,vocab_size,wordvec_size,hidden_size):
        V,D,H = vocab_size,wordvec_size,hidden_size
        rn = np.random.randn
        embed_W = (rn(V,D)/100).astype('f')
        rnn_Wx =  (rn(D,H)/np.sqrt(D)).astype('f')
        rnn_Wh =  (rn(H,H)/np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H,V)/np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.layers=[TimeEmbedding(embed_W),TimeRNN(rnn_Wx,rnn_Wh,rnn_b,True),TimeAffine(affine_W,affine_b)]

        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        self.params,self.grads = [],[]

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
    def forward(self,xs,ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs,ts)
        return loss
    def backward(self,dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
        
        
        